In [18]:
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torchvision import transforms, models
import torchvision

import time
import timeit

In [3]:
# Ensure that the output of this 2 line code is GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device.type}")

# TRAIN_DIR = "../input/cifar10-pngs-in-folders/cifar10/train"
# VALID_DIR = "../input/cifar10-pngs-in-folders/cifar10/test/"

# train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_DIR)
# valid_dataset = torchvision.datasets.ImageFolder(root=VALID_DIR)

In [4]:
TRAIN_DIR = "../input/cifar10-pngs-in-folders/cifar10/train"
VALID_DIR = "../input/cifar10-pngs-in-folders/cifar10/test/"

train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_DIR)
valid_dataset = torchvision.datasets.ImageFolder(root=VALID_DIR)

In [21]:
data_transforms = transforms.Compose(
    [#transforms.Resize([IMAGE_SIZE, IMAGE_SIZE]),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
)

BATCH_SIZE = 128

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=data_transforms)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=data_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                         shuffle=False)

In [5]:
IMAGE_SIZE = 32

gen = iter(train_dataset)

# data_transforms = transforms.Compose(
#     [#transforms.Resize([IMAGE_SIZE, IMAGE_SIZE]),
#      transforms.ToTensor(),
#      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
# )

# train_dataset = torchvision.datasets.ImageFolder(
#     root=TRAIN_DIR, transform=data_transforms
# )
# valid_dataset = torchvision.datasets.ImageFolder(
#     root=VALID_DIR, transform=data_transforms
# )
img, target = next(iter(train_dataset))
print(f"Image data type: {type(img)}")
print(f"     Image size: {img.shape}")


# BATCH_SIZE = 128

# train_loader = torch.utils.data.DataLoader(
#     train_dataset,  # our raw data
#     batch_size=BATCH_SIZE,  # the size of batches the dataloader returns
#     shuffle=True,  # shuffle our data before batching
#     drop_last=False,  # not dropping the last batch even if it's smaller than batch_size
# )

# valid_loader = torch.utils.data.DataLoader(
#     valid_dataset,  # our raw validation data
#     batch_size=BATCH_SIZE,  # the size of batches the dataloader returns
#     shuffle=True,
# )

# Method 1: SimpleCNN

In [15]:
class simple_CNN(torch.nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.main = torch.nn.Sequential(
            torch.nn.Conv2d(
                in_channels=3, out_channels=15, kernel_size=(3, 3), padding=1
            ),  # 1st CNN layer
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2, 2)),
            nn.Dropout(0.2),
            torch.nn.Conv2d(
                in_channels=15, out_channels=4, kernel_size=(3, 3), padding=1
            ),  # 2nd CNN layer
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2, 2)),
            nn.Dropout(0.2),
            torch.nn.Flatten(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 10),
        )

    def forward(self, x):
        out = self.main(x)
        return out

In [8]:
def trainer(
    model, criterion, optimizer, trainloader, validloader, epochs=5, verbose=True
):
    """Simple training wrapper for PyTorch network."""

    train_loss, valid_loss, valid_accuracy, train_accuracy = [], [], [], []
    for epoch in range(epochs):
        train_batch_loss = 0
        train_batch_acc = 0
        valid_batch_loss = 0
        valid_batch_acc = 0
        
        start = time.process_time()  
        
        # Training
        print(f"Training for epoch {epoch+1} started")
        for X, y in trainloader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()  # Zero all the gradients w.r.t. parameters
            y_hat = model(X)  # initialize model
            _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(
                1, dim=1
            )  # Assigning class label to prediction
            loss = criterion(y_hat, y)  # Calculate loss based on output
            loss.backward()  # Calculate gradients w.r.t. parameters
            optimizer.step()  # Update parameters
            train_batch_loss += loss.item()  # Add loss for this batch to running total
            train_batch_acc += (
                (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
            )
        train_loss.append(train_batch_loss / len(trainloader))
        train_accuracy.append(
            train_batch_acc / len(trainloader)
        )  # accuracy of the train set
        
        print(f"Training for epoch {epoch+1} ended in {round(time.process_time() - start, 2)} seconds")
        
        # Validation
        model.eval()
        with torch.no_grad():  # this stops pytorch doing computational graph stuff under-the-hood and saves memory and time
            for X, y in validloader:
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                _, y_hat_labels = torch.softmax(y_hat, dim=1).topk(1, dim=1)
                loss = criterion(y_hat, y)
                valid_batch_loss += loss.item()
                valid_batch_acc += (
                    (y_hat_labels.squeeze() == y).type(torch.float32).mean().item()
                )
        valid_loss.append(valid_batch_loss / len(validloader))
        valid_accuracy.append(
            valid_batch_acc / len(validloader)
        )  # accuracy of the valid set
        model.train()

        # Print progress
        if verbose:
            print(
                f"Epoch {epoch + 1}:",
                f"Train Loss: {train_loss[-1]:.3f}.",
                f"Valid Loss: {valid_loss[-1]:.3f}.",
                f"Train Accuracy: {train_accuracy[-1]:.2f}.",
                f"Valid Accuracy: {valid_accuracy[-1]:.2f}.",
            )
        print("\n")
    results = {
        "train_loss": train_loss,
        "valid_loss": valid_loss,
        "train_accuracy": train_accuracy,
        "valid_accuracy": valid_accuracy,
    }
    return results

In [19]:
torch.manual_seed(2018)

model = simple_CNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



start = timeit.default_timer()
  
results = trainer(model, criterion, optimizer, train_loader, valid_loader, epochs=15)

stop = timeit.default_timer()

print('Total Time Taken: ', stop - start)

# Densenet 121: 120 CNN layers!!!!!!!

In [6]:
densenet = models.densenet161(pretrained=True)

In [20]:
# densenet = models.densenet121(pretrained=True)

for (
    param
) in densenet.parameters():  # Freeze parameters which are not updated during training
    param.requires_grad = False
    
# Unfreeze one block(combinations of CNN layers), parameters are optimized during training using CIFAR-10 datapoints
for param in densenet.features.denseblock4.parameters():
    param.requires_grad = True
    
new_layers = nn.Sequential(
    nn.Linear(2208, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
#     nn.Linear(1024, 512),
#     nn.ReLU(),
#     nn.Linear(512, 256),
#     nn.ReLU(),
#     nn.Linear(256, 128),
#     nn.ReLU(),
#     nn.Linear(128, 64),
#     nn.ReLU(),
#     nn.Linear(64, 10)
)
densenet.classifier = new_layers

torch.manual_seed(2018)
densenet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(densenet.parameters(), lr=0.01)

start = timeit.default_timer()
results = trainer(densenet, criterion, optimizer, train_loader, valid_loader, epochs=20)
stop = timeit.default_timer()

print('Total Time Taken: ', stop - start)
# optimizer = optim.SGD(resnet.parameters(), lr=0.01, momentum=0.9, nesterov=False, weight_decay=(0.01/25))

In [13]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
test_accuracy = {}

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = densenet(images)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(labels.size(0)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        test_accuracy[classes[i]] = f"{round(acc, 2)} %"

In [14]:
pd.DataFrame(test_accuracy, index=['Accuracy']).T